In [6]:
import os
import sys
import argparse
import numpy as np
from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from MODNet.src.models.modnet import MODNet


EXTERNAL = True

if EXTERNAL:
    %store -r WIDTH
    %store -r HEIGHT
    %store -r TEXT
    %store -r VISUALIZE
    %store -r SAVE_FOLDER
    %store -r PROJECT_NAME
    %store -r SHOW_OUTPUT
    %store -r IMG_NUMBER
    %store -r THUMBNAIL
    %store -r SKIP_VIDEO
    %store -r MERGE_VIDEO
    %store -r TEXT_VOICE_GEN
    %store -r LANGUAGES
    %store -r GEN_VIDEO
    %store -r INSERT_BACKGROUND
    %store -r RECORD_FRAME_INTER_AFTER
    %store -r STATIC_BG
    %store -r LOCATION_STATIC_PNG
    %store PROJECT_BACKGROUND
else:
    WIDTH = 768
    HEIGHT = 768
    VISUALIZE = True  
    SAVE_FOLDER = "D:\\Deletar\\p_gen"
    PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
    STYLES_FOLDER = ".\\styles"
    SHOW_OUTPUT = True
    IMG_NUMBER = 2
    SKIP_VIDEO = []
    MERGE_VIDEO = False
    TEXT_VOICE_GEN = ['A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A']
    LANGUAGES = ['pt']
    GEN_VIDEO = True
    INSERT_BACKGROUND = False
    RECORD_FRAME_INTER_AFTER = 0
    RECORD_FRAME_LIP = -1
    STATIC_BG = True
    LOCATION_STATIC_PNG = "F:/gg/background/1.png"
    PROJECT_BACKGROUND = ""


project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"

input_path = 'F:/MODNet/MODNet/demo/image_matting/colab/input/output_0245 (2).png'
output_path = './out.png'
ckpt_path = './models/modnet_photographic_portrait_matting.ckpt'

class Args:
    def __init__(self):
        self.ckpt_path = ckpt_path


args = Args()

# define hyper-parameters
ref_size = 512

# define image to tensor transform
im_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

# create MODNet and load the pre-trained ckpt
modnet = MODNet(backbone_pretrained=False)
modnet = nn.DataParallel(modnet)

if torch.cuda.is_available():
    modnet = modnet.cuda()
    weights = torch.load(args.ckpt_path)
else:
    weights = torch.load(args.ckpt_path, map_location=torch.device('cpu'))
modnet.load_state_dict(weights)
modnet.eval()

DataParallel(
  (module): MODNet(
    (backbone): MobileNetV2Backbone(
      (model): MobileNetV2(
        (features): Sequential(
          (0): Sequential(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): InvertedResidual(
            (conv): Sequential(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace=True)
              (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
          (2): InvertedResidual(
            (conv): Sequential(
              (0): C

In [7]:
def generate_alpha(input_path_img, output_path_img):

    im = Image.open(input_path_img)

    # unify image channels to 3
    im = np.asarray(im)
    if len(im.shape) == 2:
        im = im[:, :, None]
    if im.shape[2] == 1:
        im = np.repeat(im, 3, axis=2)
    elif im.shape[2] == 4:
        im = im[:, :, 0:3]

    # convert image to PyTorch tensor
    im = Image.fromarray(im)
    im = im_transform(im)

    # add mini-batch dim
    im = im[None, :, :, :]

    # resize image for input
    im_b, im_c, im_h, im_w = im.shape
    if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
        if im_w >= im_h:
            im_rh = ref_size
            im_rw = int(im_w / im_h * ref_size)
        elif im_w < im_h:
            im_rw = ref_size
            im_rh = int(im_h / im_w * ref_size)
    else:
        im_rh = im_h
        im_rw = im_w
    
    im_rw = im_rw - im_rw % 32
    im_rh = im_rh - im_rh % 32
    im = F.interpolate(im, size=(im_rh, im_rw), mode='area')

    # inference
    _, _, matte = modnet(im.cuda() if torch.cuda.is_available() else im, True)

    # resize and save matte
    matte = F.interpolate(matte, size=(im_h, im_w), mode='area')
    matte = matte[0][0].data.cpu().numpy()
    Image.fromarray(((matte * 255).astype('uint8')), mode='L').save(output_path_img)

def merge_bg(lip_path_img, alpha_path_img, background_path_img, output_path_img, offset_x=0, offset_y=0):
    foreground = cv2.imread(lip_path_img)    
    alpha = cv2.imread(alpha_path_img)
    background = cv2.imread(background_path_img)

    # Add offset to foreground image
    rows, cols, channels = foreground.shape
    M = np.float32([[1, 0, offset_x], [0, 1, offset_y]])
    foreground = cv2.warpAffine(foreground, M, (cols, rows))
    alpha = cv2.warpAffine(alpha, M, (cols, rows))

    # Convert uint8 to float
    foreground = foreground.astype(float)
    background = background.astype(float)

    # Normalize the alpha mask to keep intensity between 0 and 1
    alpha = alpha.astype(float)/255

    # Multiply the foreground with the alpha matte
    foreground = cv2.multiply(alpha, foreground)

    # Multiply the background with ( 1 - alpha )
    background = cv2.multiply(1.0 - alpha, background)

    # Add the masked foreground and background.
    outImage = cv2.add(foreground, background)

    cv2.imwrite(output_path_img, outImage)




In [8]:
if EXTERNAL:
    print("Getting Alpha")
    if (not os.path.exists(f"{project_folder}//alpha")):
        os.makedirs(f"{project_folder}//alpha")
    for i, filename in enumerate(os.listdir(f"{project_folder}//lip_hd")):
        if i%50 == 0:
            print(f"file: {i}")
        img_path = f"{project_folder}//lip_hd//{filename}"
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        generate_alpha(img_path, f"{project_folder}//alpha//{filename}")
    if (not os.path.exists(f"{project_folder}//merged")):
        os.makedirs(f"{project_folder}//merged")
    print('Merging with background')
    for i, filename in enumerate(os.listdir(f"{project_folder}//lip_hd")):
        if i%50 == 0:
            print(f"file: {i}")
        if (STATIC_BG == True):
            merge_bg(f"{project_folder}//lip_hd//{filename}", f"{project_folder}//alpha//{filename}", LOCATION_STATIC_PNG, f"{project_folder}//merged//{filename}", offset_x=0, offset_y=0)
        else:
            merge_bg(f"{project_folder}//lip_hd//{filename}", f"{project_folder}//alpha//{filename}", f"F://gg//background//{PROJECT_BACKGROUND}//{filename}", f"{project_folder}//merged//{filename}", offset_x=0, offset_y=0)
else:
    generate_alpha(input_path, output_path)